# Tracing Agents

## Install Packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

In [ ]:
%pip install opentelemetry-sdk==1.34.1
%pip install azure-core-tracing-opentelemetry==1.0.0b12
%pip install azure-monitor-opentelemetry==1.6.10

## Import the libraries

In [20]:
import os, json
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import (
    FunctionTool,
    ToolSet,
    MessageRole
)
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor
from azure.ai.agents.telemetry import trace_function

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [21]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

In [22]:
# Enable Azure Monitor tracing
#application_insights_connection_string = os.environ["APPLICATIONINSIGHTS_CONNECTION_STRING"]
application_insights_connection_string = "InstrumentationKey=3b2335a3-a621-4781-bbe0-7d4b406e0445;IngestionEndpoint=https://westus-0.in.applicationinsights.azure.com/;LiveEndpoint=https://westus.livediagnostics.monitor.azure.com/;ApplicationId=d0da17ab-a9e6-47be-89c2-f32439963611"

configure_azure_monitor(connection_string=application_insights_connection_string)

scenario = "04_02 Tracing Agents.ipynb"  # Name of the current notebook
tracer = trace.get_tracer("Tracing Agents")  # Use a descriptive tracer name


## Define Functions and Function tool

In [23]:
# The trace_func decorator will trace the function call and enable adding additional attributes
# to the span in the function implementation. Note that this will trace the function parameters and their values.
@trace_function()
def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {
        "New York": "Sunny, 25°C", 
        "London": "Cloudy, 18°C", 
        "Tokyo": "Rainy, 22°C"
    }

    # Adding attributes to the current span
    span = trace.get_current_span()
    span.set_attribute("requested_location", location)

    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json

@trace_function()
def fetch_restaurant(location: str) -> str:
    """
    Fetches the restaurant information for the specified location.

    :param location (str): The location to fetch the restaurant for.
    :return: Restaurant information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a restaurant API.
    # Here, we'll mock the response.
    mock_restaurant_data = {
        "New York": "Tatiana by Kwame Onwuachi, Katz’s Delicatessen, Peter Luger Steakhouse, Sylvia's, Nathan's Famous", 
        "London": "St. JOHN, Señor Ceviche, Gloria and Circolo Popolare, Normah's, Bouchon Racine", 
        "Tokyo": "Chanko & Wanko Restaurant Asakusa Sumo Club, Sky Restaurant 634 Musashi, Ichiran, Shibuya, Rokkasen Otakibashiidori, Hakushu - Kobe Teppanyaki"
    }

    # Adding attributes to the current span
    span = trace.get_current_span()
    span.set_attribute("requested_location", location)

    restaurant = mock_restaurant_data.get(location, "Restaurant data not available for this location.")
    restaurant_json = json.dumps({"restaurant": restaurant})
    return restaurant_json

@trace_function()
def fetch_budget() -> str:
    """
    Fetches the budget information for the specified location.
    :return: budget information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a another API.
    # Here, we'll mock the response.
    mock_budget_data = {
        "New York": """
            Budget Travelers: Around $121 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $324 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $923 per day. This includes luxury hotels, fine dining, and private transportation.
        """, 
        "London": """
            Budget Travelers: Around $75 per day. This includes staying in hostels, cooking your own meals, and using public transport.
            Mid-Range Travelers: Approximately $195 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $517 per day. This includes luxury hotels, fine dining, and private transportation.
        """, 
        "Tokyo": """
            Budget Travelers: Around $100 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $286 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $908 per day. This includes luxury hotels, fine dining, and private transportation.
        """
    }
    budget_json = json.dumps({"budget": mock_budget_data})
    return budget_json

# Define user functions
user_functions = {fetch_weather, fetch_restaurant, fetch_budget}

# Initialize the FunctionTool with user-defined functions
functions = FunctionTool(functions=user_functions)
toolset = ToolSet()
toolset.add(functions)

# To enable tool calls executed automatically
project_client.agents.enable_auto_function_calls(toolset)

## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [24]:
def run_agent(user_input, thread, agent):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Tracing the application

In [25]:
with tracer.start_as_current_span(scenario):
    # Create an agent with the file search tool
    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="my-functioncalling-agent",  # Name of the agent
        instructions="You are a helpful agent and can search information from provided tools",  # Instructions for the agent
        toolset=toolset,  # Toolset containing all the tools
    
    )
    print(f"Created agent, ID: {agent.id}")

    # Create a thread for communication
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")

    for question in [
        "What is the weather in New York?",
        "What can I eat in London?",
        "With a small budget of 300USD, where can I go for 4 days?"
        ]:
        with tracer.start_as_current_span(f"run_agent_{question[:20]}"):
            run_agent(question, thread, agent)
    
    # Delete the agent after use
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")


Created agent, ID: asst_0XLoBSKEeccoLu89tK1ReNsf
Created thread, ID: thread_DNRz7RZIhnCkw8jpP3h75bmp
Created message, ID: msg_hxpAxpgvSSTVAahN8QTJ8w1j
Run finished with status: completed
Agent Response: The weather in New York is sunny with a temperature of 25°C.
Created message, ID: msg_B5rXyeJD7hnvBi0rk2FhOTIQ
Run finished with status: completed
Agent Response: In London, you can dine at the following restaurants:

1. St. JOHN
2. Señor Ceviche
3. Gloria and Circolo Popolare
4. Normah's
5. Bouchon Racine
Created message, ID: msg_bD2uApQ5yYa3S8tmeNzrI0Ih
Run finished with status: completed
Agent Response: With a small budget of 300 USD for 4 days, here are a few options:

1. **London**: 
   - Budget Travelers: Around $75 per day. This includes staying in hostels, cooking your own meals, and using public transport.
   - For 4 days: 75 * 4 = 300 USD

2. **Tokyo**:
   - Budget Travelers: Around $100 per day. This includes staying in hostels, eating at budget restaurants, and using public 